In [7]:
import scanpy as sc

In [31]:
import numpy as np
import pickle
import scanpy


def Covid_data(args):
    if args.task == 'haniffa':
        id_dict = {'Critical ': 1, 'Death': -1, 'Severe': 1, 'nan': -1, 'LPS': 0, 'Non-covid': 0, 'Asymptomatic': 1,
                   'Mild': 1, 'Healthy': 0, 'Moderate': 1}

        if args.pca == True:
            with open('./data/Haniffa/Haniffa_X_pca.npy', 'rb') as f:
                origin = np.load(f)
        else:
            with open('./data/Haniffa/origin.npy', 'rb') as f:
                origin = np.load(f)

        a_file = open('./data/Haniffa/patient_id.pkl', "rb")
        patient_id = pickle.load(a_file)
        a_file.close()

        a_file = open('./data/Haniffa/labels.pkl', "rb")
        labels = pickle.load(a_file)
        a_file.close()

        a_file = open('./data/Haniffa/cell_type.pkl', "rb")
        cell_type = pickle.load(a_file)
        a_file.close()

        a_file = open('./data/Haniffa/cell_type_large.pkl', "rb")
        cell_type_large = pickle.load(a_file)
        a_file.close()
    elif args.task == 'combat':
        id_dict = {'COVID_HCW_MILD': 1, 'COVID_CRIT': 1, 'COVID_MILD': 1, 'COVID_SEV': 1, 'COVID_LDN': 1, 'HV': 0,
                   'Flu': 0, 'Sepsis': 0}

        if args.pca == True:
            with open('./data/COMBAT/COMBAT_X_pca.npy', 'rb') as f:
                origin = np.load(f)
        else:
            with open('./data/COMBAT/origin.npy', 'rb') as f:
                origin = np.load(f)

        a_file = open('./data/COMBAT/patient_id.pkl', "rb")
        patient_id = pickle.load(a_file)
        a_file.close()

        a_file = open('./data/COMBAT/labels.pkl', "rb")
        labels = pickle.load(a_file)
        a_file.close()

        a_file = open('./data/COMBAT/cell_type.pkl', "rb")
        cell_type = pickle.load(a_file)
        a_file.close()

        a_file = open('./data/COMBAT/cell_type_large.pkl', "rb")
        cell_type_large = pickle.load(a_file)
        a_file.close()
    else:
        id_dict = {}
        if args.task == 'severity':
            id_dict = {'mild/moderate': 0, 'severe/critical': 1, 'control': -1}
        elif args.task == 'stage':
            id_dict = {'convalescence': 0, 'progression': 1, 'control': -1}

        if args.pca == True:
            with open('./data/SC4/covid_pca.npy', 'rb') as f:
                origin = np.load(f)
        else:
            with open('./data/SC4/origin.npy', 'rb') as f:
                origin = np.load(f)

        a_file = open('./data/SC4/patient_id.pkl', "rb")
        patient_id = pickle.load(a_file)
        a_file.close()

        a_file = open('./data/SC4/' + args.task + '_label.pkl', "rb")
        labels = pickle.load(a_file)
        a_file.close()

        if args.task == 'severity':
            a_file = open('./data/SC4/stage_label.pkl', "rb")
            stage_labels = pickle.load(a_file)
            a_file.close()

        a_file = open('./data/SC4/cell_type.pkl', "rb")
        cell_type = pickle.load(a_file)
        a_file.close()

        a_file = open('./data/SC4/cell_type_large.pkl', "rb")
        cell_type_large = pickle.load(a_file)
        a_file.close()

    labels_ = np.array(labels.map(id_dict))

    if args.task == 'severity':
        id_dict_ = {'convalescence': 0, 'progression': 1, 'control': 0}
        labels_stage = np.array(stage_labels.map(id_dict_))

    l_dict = {}
    indices = np.arange(origin.shape[0])
    p_ids = sorted(set(patient_id))
    p_idx = []

    if args.task == 'combat':
        top_class = []
        for tc in (
        'PB', 'CD4.TEFF.prolif', 'PLT', 'B.INT', 'CD8.TEFF.prolif', 'B.MEM', 'NK.cyc', 'RET', 'B.NAIVE', 'NK.mitohi'):
            top_class.append(indices[cell_type_large == tc])
        selected = np.concatenate(top_class)
    elif args.task == 'haniffa':
        top_class = []
        for tc in (
        'B_immature', 'C1_CD16_mono', 'CD4.Prolif', 'HSC_erythroid', 'RBC', 'Plasma_cell_IgG', 'pDC', 'Plasma_cell_IgA',
        'Platelets', 'Plasmablast'):
            top_class.append(indices[cell_type_large == tc])
        selected = np.concatenate(top_class)
    elif args.task == 'severity':
        top_class = []
        for tc in (
        'Macro_c3-EREG', 'Epi-Squamous', 'Neu_c5-GSTP1(high)OASL(low)', 'Epi-Ciliated', 'Neu_c3-CST7', 'Neu_c4-RSAD2',
        'Epi-Secretory', 'Mega', 'Neu_c1-IL1B', 'Macro_c6-VCAN', 'DC_c3-LAMP3', 'Neu_c6-FGF23', 'Macro_c2-CCL3L1',
        'Mono_c1-CD14-CCL3', 'Neu_c2-CXCR4(low)', 'B_c05-MZB1-XBP1', 'DC_c1-CLEC9A', 'Mono_c4-CD14-CD16'):
            top_class.append(indices[cell_type_large == tc])
        selected = np.concatenate(top_class)
    elif args.task == 'stage':
        top_class = []
        for tc in (
        'Neu_c5-GSTP1(high)OASL(low)', 'Neu_c3-CST7', 'Macro_c3-EREG', 'Epi-Squamous', 'Mega', 'Epi-Ciliated',
        'Mono_c5-CD16', 'Neu_c4-RSAD2', 'Epi-Secretory', 'Neu_c1-IL1B', 'DC_c1-CLEC9A', 'DC_c3-LAMP3',
        'Neu_c2-CXCR4(low)', 'Mono_c4-CD14-CD16', 'Mono_c1-CD14-CCL3', 'Macro_c6-VCAN'):
            top_class.append(indices[cell_type_large == tc])
        selected = np.concatenate(top_class)
    for i in p_ids:
        idx = indices[patient_id == i]
        if len(idx) < 500:
            continue
        if len(set(labels_[idx])) > 1:
            for ii in sorted(set(labels_[idx])):
                if ii > -1:
                    iidx = idx[labels_[idx] == ii]
                    tt_idx = iidx
                    # tt_idx = np.intersect1d(iidx, selected)
                    # tt_idx = np.setdiff1d(iidx, selected)
                    if len(tt_idx) < 1:
                        continue
                    p_idx.append(tt_idx)
                    l_dict[labels_[iidx[0]]] = l_dict.get(labels_[iidx[0]], 0) + 1
        else:
            if args.task == 'severity':
                if (labels_[idx[0]] > -1) and (labels_stage[idx[0]]) > 0:
                    tt_idx = idx
                    # tt_idx = np.intersect1d(idx, selected)
                    # tt_idx = np.setdiff1d(idx, selected)
                    if len(tt_idx) < 1:
                        continue
                    p_idx.append(tt_idx)
                    l_dict[labels_[idx[0]]] = l_dict.get(labels_[idx[0]], 0) + 1
            else:
                if labels_[idx[0]] > -1:
                    tt_idx = idx
                    # tt_idx = np.intersect1d(idx, selected)
                    # tt_idx = np.setdiff1d(idx, selected)
                    if len(tt_idx) < 1:
                        continue
                    p_idx.append(tt_idx)
                    l_dict[labels_[idx[0]]] = l_dict.get(labels_[idx[0]], 0) + 1

    # print(l_dict)

    return p_idx, labels_, cell_type, patient_id, origin, cell_type_large


def Custom_data(args):
    '''
    !!! Need to change line 178 before running the code !!!
    '''
    id_dict = {}  # {'cancer': 1, 'health': 0}
    data = scanpy.read_h5ad(args.dataset)
    if args.pca == True:
        origin = data.obsm['X_pca']
    else:
        origin = data.layers['raw']
    
    patient_id = data.obs['patient_id']

    labels = data.obs['Outcome']

    cell_type = data.obs['cell_type']

    cell_type_large = None
    # This (high resolution) cell_type is only for attention analysis, not necessary
    # cell_type_large = data.obs['cell_type_large']

    labels_ = np.array(labels.map(id_dict))

    l_dict = {}
    indices = np.arange(origin.shape[0])
    p_ids = sorted(set(patient_id))
    p_idx = []

    for i in p_ids:
        idx = indices[patient_id == i]
        if len(set(labels_[idx])) > 1:   # one patient with more than one labels
            for ii in sorted(set(labels_[idx])):
                if ii > -1:
                    iidx = idx[labels_[idx] == ii]
                    tt_idx = iidx
                    if len(tt_idx) < 500:  # exclude the sample with the number of cells fewer than 500
                        continue
                    p_idx.append(tt_idx)
                    l_dict[labels_[iidx[0]]] = l_dict.get(labels_[iidx[0]], 0) + 1
        else:
            if labels_[idx[0]] > -1:
                tt_idx = idx
                if len(tt_idx) < 500:  # exclude the sample with the number of cells fewer than 500
                    continue
                p_idx.append(tt_idx)
                l_dict[labels_[idx[0]]] = l_dict.get(labels_[idx[0]], 0) + 1

    # print(l_dict)

    return p_idx, labels_, cell_type, patient_id, origin, cell_type_large


# Sania: Redefining function for my dataset
def Custom_data(args):
    '''
    !!! Need to change line 178 before running the code !!!
    '''
    # Sania: Change this line for my datasets
    print(args)
    id_dict = {'tumor':1,'normal':0}  # {'Cancer': 1, 'health': 0}
    data = scanpy.read_h5ad("/scratch/sah2p/projects/ScRAT/data/GSE131907/04_train_clusters.h5ad")
    origin = data.obsm['X_pca']

    # if args.pca == True:
    #     origin = data.obsm['X_pca']
    # else:
    #     origin = data.layers['raw']
    
    patient_id = data.obs['sample']

    labels = data.obs['phenotype']

    cell_type = data.obs['lcell_type']

    cell_type_large = None
    # This (high resolution) cell_type is only for attention analysis, not necessary
    # cell_type_large = data.obs['cell_type_large']

    labels_ = np.array(labels.map(id_dict))

    l_dict = {}
    indices = np.arange(origin.shape[0])
    p_ids = sorted(set(patient_id))
    p_idx = []

    for i in p_ids:
        idx = indices[patient_id == i]
        if len(set(labels_[idx])) > 1:   # one patient with more than one labels
            for ii in sorted(set(labels_[idx])):
                if ii > -1:
                    iidx = idx[labels_[idx] == ii]
                    tt_idx = iidx
                    if len(tt_idx) < 500:  # exclude the sample with the number of cells fewer than 500
                        continue
                    p_idx.append(tt_idx)
                    l_dict[labels_[iidx[0]]] = l_dict.get(labels_[iidx[0]], 0) + 1
        else:
            if labels_[idx[0]] > -1:
                tt_idx = idx
                if len(tt_idx) < 500:  # exclude the sample with the number of cells fewer than 500
                    continue
                p_idx.append(tt_idx)
                l_dict[labels_[idx[0]]] = l_dict.get(labels_[idx[0]], 0) + 1

    # print(l_dict)

    return p_idx, labels_, cell_type, patient_id, origin, cell_type_large


In [32]:
#List of params from bash file
args = {}
args["model"] = "Transformer" 
args["epochs"] = 100
args["norm_first"] = False 
args["seed"] = 100
args["task"] ='custom' 
args["dataset"] ="/scratch/sah2p/projects/ScRAT/data/GSE131907/04_train_clusters.h5ad"
args["all"] = 0
args["h_dim"] = 128
args["head"] = 8
args["layers"] = 1
args["dropout"] = 0.3
args["min_size"] = 10000
args["inter_only"] = True
args["batch_size"] = 256
args["same_pheno"] = -1
args["n_splits"] = 2
args["augument_num"] = 300
args['pca']='True'
args["warmup"] = False
args["learning_rate"] = 0.01
args["alpha"] = 0.5 
args["mix_type"] = 1
args["repeat"] = 1 
args["train_sample_cells"]=500 
args["test_sample_cells"]=500  
args["train_num_sample"]=20 
args["test_num_sample"]=50

In [33]:
args

{'model': 'Transformer',
 'epochs': 100,
 'norm_first': False,
 'seed': 100,
 'task': 'custom',
 'dataset': '/scratch/sah2p/projects/ScRAT/data/GSE131907/04_train_clusters.h5ad',
 'all': 0,
 'h_dim': 128,
 'head': 8,
 'layers': 1,
 'dropout': 0.3,
 'min_size': 10000,
 'inter_only': True,
 'batch_size': 256,
 'same_pheno': -1,
 'n_splits': 2,
 'augument_num': 300,
 'pca': 'True',
 'warmup': False,
 'learning_rate': 0.01,
 'alpha': 0.5,
 'mix_type': 1,
 'repeat': 1,
 'train_sample_cells': 500,
 'test_sample_cells': 500,
 'train_num_sample': 20,
 'test_num_sample': 50}

In [34]:
p_idx, labels_, cell_type, patient_id, origin, cell_type_large = Custom_data(args)

{'model': 'Transformer', 'epochs': 100, 'norm_first': False, 'seed': 100, 'task': 'custom', 'dataset': '/scratch/sah2p/projects/ScRAT/data/GSE131907/04_train_clusters.h5ad', 'all': 0, 'h_dim': 128, 'head': 8, 'layers': 1, 'dropout': 0.3, 'min_size': 10000, 'inter_only': True, 'batch_size': 256, 'same_pheno': -1, 'n_splits': 2, 'augument_num': 300, 'pca': 'True', 'warmup': False, 'learning_rate': 0.01, 'alpha': 0.5, 'mix_type': 1, 'repeat': 1, 'train_sample_cells': 500, 'test_sample_cells': 500, 'train_num_sample': 20, 'test_num_sample': 50}


In [16]:
data = scanpy.read_h5ad("/scratch/sah2p/projects/ScRAT/data/GSE131907/04_train_clusters.h5ad")


In [41]:
data.obs

,barcode,Tissue,sample,phenotype,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,leiden_res0.01,louvain_res0.01,leiden_res0.02,louvain_res0.02,leiden_res0.05,louvain_res0.05,leiden_res0.1,louvain_res0.1,leiden_res0.5,louvain_res0.5
GACTAACCAGCTGGCT_LUNG_T31-15,GACTAACCAGCTGGCT,LUNG,T31,tumour,853,6.749931,2635,7.877018,47.703985,62.428843,...,0,0,0,0,0,0,0,0,3,4
AGAGCTTGTGAGTGAC_LUNG_N28-10,AGAGCTTGTGAGTGAC,LUNG,N28,normal,1499,7.313220,3399,8.131531,31.832892,41.806414,...,0,0,0,0,0,0,2,2,2,2
TCGGGACCAGATCCAT_LUNG_N31-2,TCGGGACCAGATCCAT,LUNG,N31,normal,2055,7.628518,7005,8.854522,47.052106,54.475375,...,1,1,1,1,1,1,1,1,5,3
CGGTTAAGTGTGCGTC_LUNG_T28-16,CGGTTAAGTGTGCGTC,LUNG,T28,tumour,1366,7.220374,4834,8.483636,39.201489,54.468349,...,1,1,1,1,1,1,1,1,4,5
GATCGATGTCTCCACT_LUNG_T31-15,GATCGATGTCTCCACT,LUNG,T31,tumour,800,6.685861,1686,7.430707,39.501779,51.720047,...,0,0,0,0,0,0,0,0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GACTAACCATTAGCCA_LUNG_T31-15,GACTAACCATTAGCCA,LUNG,T31,tumour,931,6.837333,2209,7.700748,38.569488,51.063830,...,0,0,0,0,0,0,0,0,3,4
ATAAGAGAGAGGTTGC_LUNG_T08-20,ATAAGAGAGAGGTTGC,LUNG,T08,tumour,1182,7.075809,4235,8.351375,48.288076,63.471074,...,0,0,0,0,0,0,0,0,3,4
CGGAGCTAGACCTTTG_LUNG_T06-17,CGGAGCTAGACCTTTG,LUNG,T06,tumour,1091,6.995766,2967,7.995644,43.613077,54.937647,...,0,0,0,0,0,0,0,0,1,1
CTTCTCTTCGGCTTGG_LUNG_T30-19,CTTCTCTTCGGCTTGG,LUNG,T30,tumour,1030,6.938284,3080,8.033009,44.318182,58.344156,...,4,4,4,4,4,4,5,5,8,7
